In [14]:
import pandas as pd

C = pd.read_stata(
    "/Users/clarissaache/Downloads/microdata_2005_06_social_stata/sec 2a.dta", convert_categoricals=False
)
print(C.shape)
C.sample(5)


(99315, 29)


,hhcode,province,region,psu,idc,s2aq01,s2aq02,s2aq03,s2bq01,s2bq02,...,s2bq12,s2bq13,s2bq14,s2bq15,s2bq16,s2bq17,s2bq18,s2bq19a,s2bq19b,s2bq19c
12678,1.091230e+09,1,1,10912302,2,1.0,1.0,1.0,3,NaN,...,1.0,5.0,23.0,1.0,14.0,5.0,3.0,6000.0,10000.0,16000.0
35908,1.322030e+09,1,2,13220302,52,NaN,NaN,NaN,3,NaN,...,3.0,7.0,0.0,NaN,NaN,NaN,2.0,90.0,400.0,490.0
52367,2.061320e+09,2,1,20613201,51,2.0,NaN,2.0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
31248,1.232040e+09,1,2,12320402,51,1.0,1.0,2.0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
24003,1.152020e+09,1,2,11520201,51,1.0,1.0,1.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [15]:
# Create a yes/no column for "ever admitted"
C['ever_admitted']= 'yes'
C.loc[(C["s2bq01"] == 1), 'ever_admitted'] = 'no'
C['currently_enrrolled']= 'no'
C.loc[(C["s2bq01"] == 3), 'currently_enrrolled'] = 'yes'

In [16]:
# Reduce to the questions of interest (4 questions):

C["max_level_acheived"] = C["s2bq05"]
C["why_not"] = C["s2bq02"]
#C['enrolled_last_year']=C["s2bq05"]

C_relevant = C[
    [
        "hhcode",
        "ever_admitted",
        "max_level_acheived",
        "currently_enrrolled",
        "why_not",
        "region",
        "province",
    ]
].copy()


In [17]:
C_relevant

,hhcode,ever_admitted,max_level_acheived,currently_enrrolled,why_not,region,province
0,1.011010e+09,no,NaN,no,NaN,1,1
1,1.011010e+09,yes,17.0,no,NaN,1,1
2,1.011010e+09,yes,12.0,no,NaN,1,1
3,1.011010e+09,yes,16.0,no,NaN,1,1
4,1.011010e+09,yes,NaN,yes,NaN,1,1
...,...,...,...,...,...,...,...
99310,4.071340e+09,yes,NaN,yes,NaN,1,4
99311,4.071340e+09,yes,8.0,no,NaN,1,4
99312,4.071340e+09,no,NaN,no,NaN,1,4
99313,4.071340e+09,yes,NaN,yes,NaN,1,4


In [18]:
B = pd.read_stata(
    "/Users/clarissaache/Downloads/microdata_2005_06_social_stata//1a_roster.dta"
)
B.sample(5)

,hhcode,idc,s1aq02,s1aq03,s1aq04,age,s1aq05a,s1aq05b,s1aq05c,s1aq06,s1aq07,s1aq08,s1aq09,s1aq10,region,province,psu
8606,1.061020e+09,1,head,male,1,36,1.0,7.0,1969.0,currently married,51.0,not in roster,not in roster,yes,urban,punjab,10610203
75291,3.032030e+09,53,son/daughter,female,1,17,0.0,0.0,1988.0,unmarried/never married,NaN,1,51,yes,rural,nwfp,30320301
25989,1.151240e+09,1,head,male,1,33,21.0,7.0,1972.0,currently married,51.0,not alive,55,yes,urban,punjab,11512401
83872,3.092030e+09,5,nephew/niece,male,1,16,13.0,9.0,1989.0,unmarried/never married,NaN,not alive,59,yes,rural,nwfp,30920301
13103,1.091130e+09,3,son/daughter,male,1,15,0.0,0.0,1991.0,unmarried/never married,NaN,1,51,yes,urban,punjab,10911302


In [19]:
# Change the names of the questions for readability
B['sex'] = B['s1aq03']
B['marital_status'] = B['s1aq06']

B_relevant = B[['hhcode', 'age', 'sex', 'marital_status']].copy()

In [20]:
# Made the Household Code column a str type
B["hhcode"] = B["hhcode"].astype("str")
C["hhcode"] = C["hhcode"].astype("str")

In [21]:
# MERGE
years05_06 = pd.merge(B_relevant, C_relevant, on=["hhcode"], how='inner', indicator=False)

# Evaluate merge 
# years05_06._merge.value_counts() (ALL GOOD)

In [22]:
# Select people in school enrollment age (I would say 4-10 years old)
years05_06 = years05_06.loc[(years05_06["age"] >= 4)&(years05_06["age"] <= 15)]

In [23]:
years05_06.province.value_counts()
# weird! It doesnt have the provinces with codes 5,6,7

1    104188
3     83419
2     76920
4     54703
Name: province, dtype: int64

In [24]:
# Lets see which hh codes we have:
years05_06["hhcode"] = years05_06["hhcode"].astype("str")
years05_06['subprovince code']= years05_06['hhcode'].apply(lambda x: x[0:4])
years05_06["subprovince code"] = years05_06["subprovince code"].astype("int")
years05_06.loc[(years05_06["region"] == 1), 'region'] = 'urban'
years05_06.loc[(years05_06["region"] == 2), 'region'] = 'rural'

In [25]:
# Now we import the stratum name - code data
strat_name = pd.read_csv(
    "/Users/clarissaache/Documents/IDS 701/stratum0506_v1.csv"
)
strat_name.sample(5)

,province,subprovince code,subprovince
94,Balochistan,4212.0,4212
92,Balochistan,4192.0,4192
11,Punjab,1122.0,Jhang
81,Balochistan,4082.0,4082
122,Sindh,2011.0,Sukkar


In [26]:
# MERGE
years05_06_ = pd.merge(years05_06, strat_name, on=["subprovince code"], how='inner', indicator=True)

# Evaluate merge 
years05_06_._merge.value_counts()

both          314422
left_only          0
right_only         0
Name: _merge, dtype: int64

In [27]:
years05_06_

,hhcode,age,sex,marital_status,ever_admitted,max_level_acheived,currently_enrrolled,why_not,region,province_x,subprovince code,province_y,subprovince,_merge
0,1011010102.0,8,male,unmarried/never married,yes,12.0,no,NaN,urban,1,1011,NaN,Rawalpindi,both
1,1011010102.0,8,male,unmarried/never married,yes,16.0,no,NaN,urban,1,1011,NaN,Rawalpindi,both
2,1011010102.0,8,male,unmarried/never married,yes,NaN,yes,NaN,urban,1,1011,NaN,Rawalpindi,both
3,1011010102.0,8,male,unmarried/never married,yes,NaN,yes,NaN,urban,1,1011,NaN,Rawalpindi,both
4,1011010102.0,6,female,unmarried/never married,yes,12.0,no,NaN,urban,1,1011,NaN,Rawalpindi,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314417,4071340112.0,5,male,unmarried/never married,yes,NaN,yes,NaN,urban,4,4071,NaN,Quetta,both
314418,4071340112.0,4,female,unmarried/never married,yes,8.0,no,NaN,urban,4,4071,NaN,Quetta,both
314419,4071340112.0,4,female,unmarried/never married,no,NaN,no,NaN,urban,4,4071,NaN,Quetta,both
314420,4071340112.0,4,female,unmarried/never married,yes,NaN,yes,NaN,urban,4,4071,NaN,Quetta,both


In [28]:
years05_06_.to_csv('/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/10_data_cleaning/clean_clarissa/years05_06_.', index_label="id")